# SQL Part 2: Tables, Procedures & Token Generation

**Continue from the beginner guide!** This notebook covers:
- Working with tables (storing data)
- Creating stored procedures
- Error handling
- Building the complete `mobile_bearer_token_get` procedure

**Make sure you completed the beginner guide first!**

## Lesson 8: Creating Tables to Store Data

Tables are like Excel spreadsheets - they have columns and rows to store information.

In [ ]:
-- Create a simple table for practice
CREATE TABLE Students (
    ID INT IDENTITY(1,1) PRIMARY KEY,  -- Auto-numbering: 1, 2, 3, 4...
    Name VARCHAR(100),                  -- Student name (up to 100 characters)
    Age INT,                           -- Student age
    Grade VARCHAR(2),                  -- Grade like 'A', 'B', 'C'
    EnrollDate DATE DEFAULT GETDATE()  -- When they enrolled (defaults to today)
);

-- Create our token storage table (like in the original procedure)
CREATE TABLE TokenMaster (
    ID INT IDENTITY(1,1) PRIMARY KEY,
    token_id VARCHAR(32) NOT NULL,     -- The actual token
    created_datetime DATETIME2 DEFAULT GETDATE(),
    expire_datetime DATETIME2 DEFAULT DATEADD(HOUR, 24, GETDATE()),  -- Expires in 24 hours
    is_active BIT DEFAULT 1            -- 1 = active, 0 = inactive
);

SELECT 'Tables created successfully!' AS Message;

## Lesson 9: Adding Data to Tables (INSERT)

Learn to put information into your tables.

In [ ]:
-- Add students one by one
INSERT INTO Students (Name, Age, Grade)
VALUES ('John Smith', 20, 'A');

INSERT INTO Students (Name, Age, Grade)
VALUES ('Jane Doe', 19, 'B');

-- Add multiple students at once
INSERT INTO Students (Name, Age, Grade)
VALUES 
    ('Bob Johnson', 21, 'A'),
    ('Alice Brown', 18, 'C'),
    ('Charlie Wilson', 22, 'B');

-- See what we added
SELECT * FROM Students;

-- Add some test tokens
INSERT INTO TokenMaster (token_id)
VALUES ('ABC123DEF456');

INSERT INTO TokenMaster (token_id, expire_datetime)
VALUES ('OLD123TOKEN', DATEADD(HOUR, -1, GETDATE()));  -- This one is already expired

-- See our tokens
SELECT * FROM TokenMaster;

## Lesson 10: Getting Data from Tables (SELECT with WHERE)

Learn to find specific information in your tables.

In [ ]:
-- Get all students
SELECT * FROM Students;

-- Get only students with grade 'A'
SELECT * FROM Students WHERE Grade = 'A';

-- Get students older than 20
SELECT * FROM Students WHERE Age > 20;

-- Get students whose name contains 'John'
SELECT * FROM Students WHERE Name LIKE '%John%';

-- Count how many students we have
SELECT COUNT(*) AS TotalStudents FROM Students;

-- Get only active tokens
SELECT * FROM TokenMaster WHERE is_active = 1;

-- Get expired tokens
SELECT * FROM TokenMaster WHERE expire_datetime < GETDATE();

-- Check if we have any tokens with specific value
SELECT COUNT(*) AS TokenExists 
FROM TokenMaster 
WHERE token_id = 'ABC123DEF456' AND is_active = 1;

## Lesson 11: Changing and Removing Data (UPDATE and DELETE)

Learn to modify existing data or remove it completely.

In [ ]:
-- Update a student's grade
UPDATE Students 
SET Grade = 'A+' 
WHERE Name = 'Jane Doe';

-- Update multiple columns
UPDATE Students 
SET Age = 23, Grade = 'A' 
WHERE Name = 'Charlie Wilson';

-- See the changes
SELECT * FROM Students;

-- Deactivate expired tokens (don't delete, just mark inactive)
UPDATE TokenMaster 
SET is_active = 0 
WHERE expire_datetime < GETDATE();

-- See the token changes
SELECT * FROM TokenMaster;

-- Delete expired and inactive tokens completely
DELETE FROM TokenMaster 
WHERE expire_datetime < GETDATE() AND is_active = 0;

-- Check what's left
SELECT * FROM TokenMaster;

-- Important: @@ROWCOUNT tells us how many rows were affected
DELETE FROM Students WHERE Age > 25;
SELECT @@ROWCOUNT AS RowsDeleted;

## Lesson 12: Your First Stored Procedure

Procedures are like recipes - they contain a series of steps that can be run anytime.

In [ ]:
-- Create a simple procedure
CREATE PROCEDURE GetAllStudents
AS
BEGIN
    SELECT * FROM Students ORDER BY Name;
END;
GO

-- Run the procedure
EXEC GetAllStudents;

-- Create a procedure that takes input
CREATE PROCEDURE GetStudentsByGrade
    @Grade VARCHAR(2)  -- Input parameter
AS
BEGIN
    SELECT * FROM Students WHERE Grade = @Grade;
END;
GO

-- Run with different inputs
EXEC GetStudentsByGrade @Grade = 'A';
EXEC GetStudentsByGrade @Grade = 'B';

-- Create a procedure that returns information through output parameters
CREATE PROCEDURE CountStudents
    @TotalCount INT OUTPUT,  -- Output parameter
    @AverageAge DECIMAL(5,2) OUTPUT
AS
BEGIN
    SELECT 
        @TotalCount = COUNT(*),
        @AverageAge = AVG(CAST(Age AS DECIMAL(5,2)))
    FROM Students;
END;
GO

-- Test the output procedure
DECLARE @Count INT, @AvgAge DECIMAL(5,2);

EXEC CountStudents 
    @TotalCount = @Count OUTPUT,
    @AverageAge = @AvgAge OUTPUT;

SELECT 
    @Count AS TotalStudents,
    @AvgAge AS AverageAge;

## Lesson 13: Error Handling in Procedures

Learn to handle errors gracefully so your procedures don't crash.

In [ ]:
-- Create a procedure with error handling
CREATE PROCEDURE AddStudent
    @Name VARCHAR(100),
    @Age INT,
    @Grade VARCHAR(2),
    @StatusCode INT OUTPUT,
    @StatusMessage VARCHAR(200) OUTPUT
AS
BEGIN
    -- Initialize outputs
    SET @StatusCode = 0;
    SET @StatusMessage = '';
    
    BEGIN TRY
        -- Validate inputs
        IF @Name IS NULL OR @Name = ''
        BEGIN
            SET @StatusCode = 400;
            SET @StatusMessage = 'Name is required';
            RETURN;
        END
        
        IF @Age < 0 OR @Age > 150
        BEGIN
            SET @StatusCode = 400;
            SET @StatusMessage = 'Age must be between 0 and 150';
            RETURN;
        END
        
        -- Insert the student
        INSERT INTO Students (Name, Age, Grade)
        VALUES (@Name, @Age, @Grade);
        
        -- Check if insert was successful
        IF @@ROWCOUNT > 0
        BEGIN
            SET @StatusCode = 200;
            SET @StatusMessage = 'Student added successfully';
        END
        ELSE
        BEGIN
            SET @StatusCode = 500;
            SET @StatusMessage = 'Failed to add student';
        END
        
    END TRY
    BEGIN CATCH
        SET @StatusCode = 500;
        SET @StatusMessage = 'Error: ' + ERROR_MESSAGE();
    END CATCH
END;
GO

-- Test the error handling
DECLARE @Code INT, @Message VARCHAR(200);

-- Test with valid data
EXEC AddStudent 
    @Name = 'Sarah Connor',
    @Age = 25,
    @Grade = 'A',
    @StatusCode = @Code OUTPUT,
    @StatusMessage = @Message OUTPUT;

SELECT @Code AS StatusCode, @Message AS StatusMessage;

-- Test with invalid data
EXEC AddStudent 
    @Name = '',
    @Age = 25,
    @Grade = 'A',
    @StatusCode = @Code OUTPUT,
    @StatusMessage = @Message OUTPUT;

SELECT @Code AS StatusCode, @Message AS StatusMessage;

## Lesson 14: Building a Simple Token Generator

Now let's combine everything to create a basic token generation procedure.

In [ ]:
-- Create a simple token generation procedure
CREATE PROCEDURE GenerateSimpleToken
    @TokenLength INT = 16,
    @GeneratedToken VARCHAR(50) OUTPUT,
    @StatusCode INT OUTPUT,
    @StatusMessage VARCHAR(200) OUTPUT
AS
BEGIN
    -- Initialize variables
    DECLARE @NewToken VARCHAR(50) = '';
    DECLARE @Characters VARCHAR(62) = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789';
    DECLARE @i INT = 1;
    
    BEGIN TRY
        -- Validate input
        IF @TokenLength <= 0 OR @TokenLength > 50
        BEGIN
            SET @GeneratedToken = '';
            SET @StatusCode = 400;
            SET @StatusMessage = 'Token length must be between 1 and 50';
            RETURN;
        END
        
        -- Generate random token
        WHILE @i <= @TokenLength
        BEGIN
            SET @NewToken = @NewToken + SUBSTRING(@Characters, ABS(CHECKSUM(NEWID())) % LEN(@Characters) + 1, 1);
            SET @i = @i + 1;
        END
        
        -- Store in database
        INSERT INTO TokenMaster (token_id)
        VALUES (@NewToken);
        
        -- Check if successful
        IF @@ROWCOUNT > 0
        BEGIN
            SET @GeneratedToken = @NewToken;
            SET @StatusCode = 200;
            SET @StatusMessage = 'Token generated successfully';
        END
        ELSE
        BEGIN
            SET @GeneratedToken = '';
            SET @StatusCode = 500;
            SET @StatusMessage = 'Failed to store token';
        END
        
    END TRY
    BEGIN CATCH
        SET @GeneratedToken = '';
        SET @StatusCode = 500;
        SET @StatusMessage = 'Error: ' + ERROR_MESSAGE();
    END CATCH
END;
GO

-- Test the token generator
DECLARE @Token VARCHAR(50), @Code INT, @Message VARCHAR(200);

EXEC GenerateSimpleToken
    @TokenLength = 16,
    @GeneratedToken = @Token OUTPUT,
    @StatusCode = @Code OUTPUT,
    @StatusMessage = @Message OUTPUT;

SELECT 
    @Token AS GeneratedToken,
    @Code AS StatusCode,
    @Message AS StatusMessage;

-- See all tokens
SELECT * FROM TokenMaster ORDER BY created_datetime DESC;

## Lesson 15: The Complete mobile_bearer_token_get Procedure

Now we'll build the exact procedure from your CERS application!

In [ ]:
-- This is the EXACT same logic as your mobile_bearer_token_get procedure!
CREATE PROCEDURE mobile_bearer_token_get
    @status_code INT = 0 OUTPUT,
    @status_message VARCHAR(200) = '' OUTPUT
AS
BEGIN
    -- Declare variables (exactly like the original)
    DECLARE @token VARCHAR(32) = '';
    DECLARE @char_set VARCHAR(75) = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789';
    DECLARE @Length INT = 32;
    DECLARE @token_id VARCHAR(32) = '';
    DECLARE @i INT = 1;
    
    BEGIN TRY
        -- Generate random token using WHILE loop (same as original)
        WHILE @i <= @Length
        BEGIN
            SET @token_id = @token_id + SUBSTRING(@char_set, ABS(CHECKSUM(NEWID())) % LEN(@char_set) + 1, 1);
            SET @i = @i + 1;
        END
        
        -- Clean up expired tokens (same as original)
        DELETE FROM TokenMaster 
        WHERE expire_datetime < GETDATE();
        
        -- Insert new token (same as original)
        INSERT INTO TokenMaster (token_id) 
        VALUES (@token_id);
        
        -- Check if insertion was successful (same logic as original)
        IF @@ROWCOUNT > 0
        BEGIN
            SELECT @status_code = 200;
            SELECT @status_message = 'Created';
        END
        ELSE
        BEGIN
            SELECT @token_id = '';
            SELECT @status_code = 400;
            SELECT @status_message = 'Token Generation Failed';
        END
        
    END TRY
    BEGIN CATCH
        -- Handle errors
        SELECT @token_id = '';
        SELECT @status_code = 500;
        SELECT @status_message = 'Internal Error: ' + ERROR_MESSAGE();
    END CATCH
    
    -- Return results (same format as original)
    SELECT 
        @token_id AS token_id,
        @status_code AS status_code,
        @status_message AS status_message;
END;
GO

-- Test the complete procedure
DECLARE @code INT, @message VARCHAR(200);

EXEC mobile_bearer_token_get
    @status_code = @code OUTPUT,
    @status_message = @message OUTPUT;

PRINT 'Status Code: ' + CAST(@code AS VARCHAR(10));
PRINT 'Status Message: ' + @message;

-- Run it multiple times to see different tokens
EXEC mobile_bearer_token_get;
EXEC mobile_bearer_token_get;
EXEC mobile_bearer_token_get;

-- See all generated tokens
SELECT * FROM TokenMaster ORDER BY created_datetime DESC;

## 🎉 Congratulations!

You've just learned to write the complete `mobile_bearer_token_get` procedure!

**What you've mastered:**
- ✅ Basic SQL commands
- ✅ Variables and data types
- ✅ IF statements and WHILE loops
- ✅ Random number generation
- ✅ String manipulation
- ✅ Creating and working with tables
- ✅ INSERT, UPDATE, DELETE operations
- ✅ Stored procedures with parameters
- ✅ Error handling with TRY-CATCH
- ✅ Output parameters and status codes
- ✅ The complete token generation procedure

**Practice Exercises:**
1. Modify the procedure to generate tokens of different lengths
2. Add a parameter to set custom expiration times
3. Create a procedure to validate existing tokens
4. Add logging to track token generation
5. Create procedures for other CERS operations

**You're now ready to understand and modify any SQL procedure in the CERS application!**